In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, json, requests
from pathlib import Path
# from requests.auth import HTTPBasicAuth

import time, datetime

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt 
import subprocess
from shapely.geometry import shape, mapping
from planet import api

import ipdb
from pprint import pprint
from IPython.display import JSON, GeoJSON

from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)


/home/hayley/miniconda3/envs/widgets/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/hayley/miniconda3/envs/widgets/lib/python3.6/site-packages/pkg_resources/__init__.py:1151: DeprecationWarning: Use of .. or absolute path in a resource path is not allowed and will raise exceptions in a future release.
  self, resource_name
/home/hayley/miniconda3/envs/widgets/lib/python3.6/site-packages/branca/colormap.py:29: ResourceWarning: unclosed file <_io.BufferedReader name='/home/hayley/miniconda3/envs/widgets/lib/python3.6/site-packages/branca/_cnames.json'>
  resource_package, resource_path_cnames).read().decode()
/home/hayley/miniconda3/envs/widgets/lib/python3.6/site-packages/branca/colormap.py:33: ResourceWarning: unclosed file <_io.BufferedReader name='/home/hayley/miniconda3/envs/widgets/lib/python3.6/site-packages/branca/_schemes.json'>
  resource_package,

In [3]:
def get_mro(someobj):
    mro_tuple = someobj.__class__.__mro__
    display(mro_tuple)
    return mro_tuple

In [4]:
my_key = '81c0dedd35fd4a398da77753b90d62ea' # songcoco15
# my_key = '7d8af35b6e944f33bb5e33ada32ab4a0' #hayleysong15

# Set up the Planet API client - method1
os.environ['PL_API_KEY']  = my_key
client = api.ClientV1()

# Method2: alternatively, 
# client = api.ClientV1(api_key=my_key)

In [5]:
# Basemap Mosaic (v1 API)
# mosaic_name = 'global_quarterly_2017q1_mosaic'
mosaic_name = 'global_monthly_2016_05_mosaic'


# Planet tile server base URL (Planet Explorer Mosaics Tiles)
# tiles_url_base = f'https://tiles0.planet.com/experimental/mosaics/planet-tiles/{mosaic_name}' + \
#                         '/gmap/{z}/{x}/{y}.png'

tiles_url_base = f"https://tiles.planet.com/basemaps/v1/planet-tiles/{mosaic_name}" + \
                      "/gmap/{z}/{x}/{y}.png"
temp = "https://tiles.planet.com/basemaps/v1/planet-tiles/global_monthly_2019_01_mosaic/gmap/0/0/0.png"
# temp = https://tiles.planet.com/basemaps/v1/planet-tiles/global_monthly_2018_01_mosaic/gmap/0/0/0.png?api_key=81c0dedd35fd4a398da77753b90d62ea
# Planet tile server url
tiles_url = f'{tiles_url_base}?api_key={my_key}'

print(tiles_url)
print(tiles_url.format({z:0, x:0, y:0}))

https://tiles.planet.com/basemaps/v1/planet-tiles/global_monthly_2016_05_mosaic/gmap/{z}/{x}/{y}.png?api_key=81c0dedd35fd4a398da77753b90d62ea


NameError: name 'z' is not defined

In [6]:
# Map Settings 
# Define colors
colors = {'blue': "#009da5"}
# Define initial map center lat/long
paris_latlon = [48.864716, 2.349014] 
jeonju_latlon = [35.8242, 127.1480] 
naju_latlon = [35.032194, 126.715671]
# Define initial map zoom level
zoom = 15
# Set Map Tiles URL
tile_layer = TileLayer(url= tiles_url)

In [7]:
# Create the map
m = Map(
    center=naju_latlon, 
    zoom=zoom,
#     default_tiles = tile_layer # Uncomment to use Planet.com basemap
)
m.layout={'height': '500px'}
# m.add_layer(tile_layer)
# display(m)

In [8]:
# Define an output widget to capture the handler's intermediate printouts
import ipywidgets as widgets
out = widgets.Output(layout={'border': '1px solid black'})
# out

In [9]:
# Define the draw tool type options
polygon = {'shapeOptions': {'color': colors['blue']}}
rectangle = {'shapeOptions': {'color': colors['blue']}} 

# Create the draw controls
# @see https://github.com/ellisonbg/ipyleaflet/blob/master/ipyleaflet/leaflet.py#L293
dc = DrawControl(
    polygon = polygon,
    rectangle = rectangle
)

# Initialize an action counter variable
actionCount = 0
AOIs = {}
out.clear_output()

# Register the draw controls handler
@out.capture(clear_output=False)
def handle_draw(self, action, geo_json):
    # Increment the action counter
    global actionCount
    
    # Remove the `style` property from the GeoJSON
    geo_json['properties'] = {}
    # Convert geo_json output to a string and prettify (indent & replace ' with ")
    geojsonStr = json.dumps(geo_json, indent=2).replace("'", '"')
    AOIs[actionCount] = json.loads(geojsonStr)
    
    # print to out widget
    print("="*80)
    print("handle_draw called")
    print('\tcurr actionCount: ', actionCount)
    print('\taction: ', action)
    print('\tgeo_json: ');pprint(geo_json)
    print('AOIs')
    pprint(AOIs)
    
    actionCount += 1

# Attach the draw handler to the draw controls `on_draw` event
dc.on_draw(handle_draw)
m.add_control(dc)
display(m)
display(out)

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

Output(layout=Layout(border='1px solid black'))